# 台灣加權股價指數預測 - 使用 AutoTS
> 利用 Python 強大的時間序列分析的 AutoTS 函式庫進行台灣加權股價指數預測

- toc: false
- badges: false
- comments: false
- sticky_rank: 1
- author: ChiHong Lin
- categories: [python, prediction, jupyter, AutoTS, Stock Price]

#### 股價預測是機器學習在金融領域最重要的應用之一。在這篇文章中，我將嘗試透過使用 Python 的線性回歸模型來進行股票價格的預測。

預測股市的價格，一直是投資者的終極目標。 在每天數以億計的交易之中，每筆交易，都代表者投資者對於該股票的價格預期，並且期望透過交易獲利。

也因此，股票漲跌，便取決於投資者在交易市場中的投資行為。 如果投資者能夠準確預測市場動向，便有機會創造誘人的財富。

如果您具有股票市場的投資經驗以及機器學習的量化數據分析技能，對於您預測價格趨勢將會有明顯的助益。

## Python 函式庫 AutoTS

AutoTS 是一個用於自動時間序列分析的 Python 函式庫。它是 autoML 的一部分，其目標是為初學者提供自動化的時間序列模型工具。

時間序列問題無論是在銷量預測，天氣預測還是在股票預測等問題中都至關重要，而如今隨著機器學習等快速發展，已經出現了非常多時間序列建模相關的函式庫，AutoTS 融合了自動化機器學習技術，會先對數據進行預處理，從數據中刪除異常值，再通過學習尋找最佳值。Auto 只需使用一行代碼，就可以訓練多個時間序列模型，包括ARIMA、SARIMAX、FB Prophet、VAR，並得出效果最佳的模型。它具有以下幾個特性：

1. 它可用於查找最佳時間序列預測模型，具體取決於您使用的數據類型。
2. 它可以處理單變量和多變量時間序列。
3. 它還可以通過刪除和填充 NaN 值來處理混亂的數據，它還可以處理異常值。

您可以將此庫用於時間序列預測的任何任務，例如預測未來 n 天的股票價格。

AutoTS是一個非常不錯的時間序列函式庫，在碰到時間序列問題時，可以考慮使用AutoTS來進行訓練和預測，作為一個非常不錯的Baseline。

### 參考連結

- https://pypi.org/project/AutoTS/
- https://github.com/winedarksea/AutoTS

## 台灣加權股價指數預測

我將透過 [yfinance API](https://pypi.org/project/yfinance/) 從 [yfinance API](https://pypi.org/project/yfinance/) 取得台灣加權指數（代碼 **^TWII**）近3年的歷史價格數據，並藉由這些數據來進行分析及預測。

我們首先先將需要的函式庫載入：

In [2]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta

import warnings
warnings.filterwarnings("ignore")

透過 [yfinance API](https://pypi.org/project/yfinance/) 從 [yfinance API](https://pypi.org/project/yfinance/) 取得台灣加權指數（代碼 **^TWII**）近3年的歷史價格數據

In [3]:
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=1095)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('^TWII',
                   start=start_date,
                   end=end_date,
                   progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)

初步了解取得的數據內容

In [4]:
print(data.shape)
print(data.head())

(730, 7)
        Date          Open          High           Low         Close  \
0 2019-06-17  10488.700195  10562.969727  10474.190430  10530.540039   
1 2019-06-18  10547.150391  10573.769531  10521.700195  10566.740234   
2 2019-06-19  10650.480469  10778.639648  10650.480469  10775.339844   
3 2019-06-20  10749.410156  10799.139648  10745.250000  10785.009766   
4 2019-06-21  10817.709961  10840.290039  10773.469727  10803.769531   

      Adj Close   Volume  
0  10530.540039  1444800  
1  10566.740234  1524200  
2  10775.339844  2370300  
3  10785.009766  2079800  
4  10803.769531  2780700  


我們繪製一下指數的走勢（以K線圖表示）：

In [6]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=data["Close"])])
figure.update_layout(title = "台灣加權指數走勢", 
                     xaxis_rangeslider_visible=False)
figure.show()

分析時間序列數據的最佳方法之一是創建互動式圖表，您可以在其中手動選擇輸出圖表本身的時間間隔。
一種方法是在圖表下方添加一個滑動區塊，並在圖表上方添加按鈕來控制時間的間隔。 以下是創建互動式K線圖表的方法，您可以在其中選擇輸出本身的時間間隔：

In [8]:
figure = go.Figure(data = [go.Candlestick(x = data.index,
                                          open = data["Open"],
                                          high = data["High"],
                                          low = data["Low"],
                                          close = data["Close"])])
figure.update_layout(title = "台灣加權指數走勢 (互動式圖表)")

figure.update_xaxes(
    rangeslider_visible = True,
    rangeselector = dict(
        buttons = list([
            dict(count = 1, label = "1m", step = "month", stepmode = "backward"),
            dict(count = 6, label = "6m", step = "month", stepmode = "backward"),
            dict(count = 1, label = "YTD", step = "year", stepmode = "todate"),
            dict(count = 1, label = "1y", step = "year", stepmode = "backward"),
            dict(step = "all")
        ])
    )
)
figure.show()

## AutoTS 時間序列下的價格預測

接下來，我們透過AutoTS函式庫，來預測接下來30天的台灣加權股價指數（收盤價格）：

In [9]:
from autots import AutoTS  # 載入 AutoTS 函式庫
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast

Inferred frequency is: B
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


C:\Users\impep\anaconda3\envs\OpenCV\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Model Number: 6 with model DatepartRegression in generation 0 of 10


C:\Users\impep\anaconda3\envs\OpenCV\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



Model Number: 7 with model DatepartRegression in generation 0 of 10
Epoch 1/50
24/24 [==============================] - 9s 8ms/step - loss: 0.4147
Epoch 2/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4123
Epoch 3/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4116
Epoch 4/50
24/24 [==============================] - 0s 7ms/step - loss: 0.4103
Epoch 5/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4097
Epoch 6/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4109
Epoch 7/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4109
Epoch 8/50
24/24 [==============================] - 0s 7ms/step - loss: 0.4082
Epoch 9/50
24/24 [==============================] - 0s 8ms/step - loss: 0.4064
Epoch 10/50
24/24 [==============================] - 0s 9ms/step - loss: 0.4070
Epoch 11/50
24/24 [==============================] - 0s 9ms/step - loss: 0.4073
Epoch 12/50
24/24 [==============================] - 0s 7ms/s

...

94 - WindowRegression with avg smape 1.52: 
Model Number: 95 of 130 with model WindowRegression for Validation 3
95 - WindowRegression with avg smape 2.21: 
Model Number: 96 of 130 with model MultivariateRegression for Validation 3
96 - MultivariateRegression with avg smape 10.31: 
Model Number: 97 of 130 with model GLS for Validation 3
97 - GLS with avg smape 1.47: 
Model Number: 98 of 130 with model WindowRegression for Validation 3
98 - WindowRegression with avg smape 1.98: 
Model Number: 99 of 130 with model ETS for Validation 3
99 - ETS with avg smape 1.68: 
Model Number: 100 of 130 with model SectionalMotif for Validation 3
100 - SectionalMotif with avg smape 2.71: 
Model Number: 101 of 130 with model UnobservedComponents for Validation 3
101 - UnobservedComponents with avg smape 1.46: 
Model Number: 102 of 130 with model WindowRegression for Validation 3
102 - WindowRegression with avg smape 1.81: 
Model Number: 103 of 130 with model SeasonalNaive for Validation 3
103 - Seasonal

C:\Users\impep\anaconda3\envs\OpenCV\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.126e+06, tolerance: 4.854e+05



110 - WindowRegression with avg smape 1.3: 
Model Number: 111 of 130 with model MultivariateMotif for Validation 3
111 - MultivariateMotif with avg smape 12.3: 
Model Number: 112 of 130 with model UnivariateMotif for Validation 3
112 - UnivariateMotif with avg smape 1.51: 
Model Number: 113 of 130 with model MultivariateRegression for Validation 3
113 - MultivariateRegression with avg smape 1.11: 
Model Number: 114 of 130 with model MultivariateMotif for Validation 3
114 - MultivariateMotif with avg smape 13.83: 
Model Number: 115 of 130 with model UnivariateMotif for Validation 3
115 - UnivariateMotif with avg smape 12.28: 
Model Number: 116 of 130 with model AverageValueNaive for Validation 3
116 - AverageValueNaive with avg smape 8.7: 
Model Number: 117 of 130 with model GLS for Validation 3
117 - GLS with avg smape 1.4: 
Model Number: 118 of 130 with model AverageValueNaive for Validation 3
118 - AverageValueNaive with avg smape 1.57: 
Model Number: 119 of 130 with model Multivaria

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_

119 - MultivariateRegression with avg smape 1.76: 
Model Number: 120 of 130 with model WindowRegression for Validation 3
Epoch 1/50
17/17 [==============================] - 11s 116ms/step - loss: 105.7642 - val_loss: 108.1195
Epoch 2/50
17/17 [==============================] - 0s 25ms/step - loss: 101.8077 - val_loss: 108.1208
Epoch 3/50
17/17 [==============================] - 1s 32ms/step - loss: 111.3971 - val_loss: 108.1320
Epoch 4/50
17/17 [==============================] - 0s 26ms/step - loss: 102.1923 - val_loss: 108.1304
Epoch 5/50
17/17 [==============================] - 0s 27ms/step - loss: 105.6344 - val_loss: 108.1349
Epoch 6/50
17/17 [==============================] - 1s 41ms/step - loss: 103.9355 - val_loss: 108.1401
Epoch 7/50
17/17 [==============================] - 1s 51ms/step - loss: 98.0228 - val_loss: 108.1465
Epoch 8/50
17/17 [==============================] - 1s 63ms/step - loss: 115.9061 - val_loss: 108.1488
Epoch 9/50
17/17 [==============================] - 1s

在經過漫長（但自動化）的複雜運算之後，AutoTS會選擇最佳的模型預測結果

In [10]:
print(forecast)

                   Close
2022-06-16  16021.916427
2022-06-17  16009.360169
2022-06-20  15971.442562
2022-06-21  15958.721237
2022-06-22  15945.959265
2022-06-23  15933.156894
2022-06-24  15920.314374
2022-06-27  15881.548438
2022-06-28  15868.547841
2022-06-29  15855.508357
2022-06-30  15842.430242
2022-07-01  15829.313751
2022-07-04  15789.736584
2022-07-05  15776.469157
2022-07-06  15763.164642
2022-07-07  15749.823298
2022-07-08  15736.445387
2022-07-11  15696.094866
2022-07-12  15682.573305
2022-07-13  15669.016490
2022-07-14  15655.424686
2022-07-15  15641.798158
2022-07-18  15600.712897
2022-07-19  15586.950142
2022-07-20  15573.154002
2022-07-21  15559.324744
2022-07-22  15545.462640
2022-07-25  15503.681958
2022-07-26  15489.691181
2022-07-27  15475.668918


從預測數據來看，在未來30天，台灣加權股價指數為向下趨勢。

## 總結

投資買賣一定會帶來風險。 使用機器學習進行加價格預測，是根據歷史價格進行推算所得到的結果，希望您在投資之前先做好所有的風險評估。
